In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/Weather_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,81.95,79,14,18.77,light rain
1,1,Punta Arenas,CL,-53.1500,-70.9167,35.71,64,75,4.61,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,35.26,69,75,14.97,broken clouds
3,3,Najran,SA,17.4924,44.1277,88.45,26,100,8.32,light rain
4,4,Lompoc,US,34.6391,-120.4579,67.93,84,100,11.01,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,81.95,79,14,18.77,light rain
3,3,Najran,SA,17.4924,44.1277,88.45,26,100,8.32,light rain
4,4,Lompoc,US,34.6391,-120.4579,67.93,84,100,11.01,overcast clouds
9,9,Rikitea,PF,-23.1203,-134.9692,72.23,80,22,7.43,light rain
11,11,Bambous Virieux,MU,-20.3428,57.7575,70.21,68,75,18.41,broken clouds
12,12,Mahebourg,MU,-20.4081,57.7000,71.26,68,75,18.41,broken clouds
16,16,Puerto Colombia,CO,10.9878,-74.9547,87.89,74,100,3.40,overcast clouds
18,18,Ponta Do Sol,PT,32.6667,-17.1000,78.12,61,4,3.65,clear sky
19,19,Kapaa,US,22.0752,-159.3190,75.18,93,76,3.00,broken clouds
22,22,Cravo Norte,CO,6.3017,-70.2041,85.15,65,83,4.43,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.isnull().sum()
preferred_cities_df.count()

Unnamed: 0             440
City                   440
Country                439
Lat                    440
Lng                    440
Max Temp               440
Humidity               440
Cloudiness             440
Wind Speed             440
Current Description    440
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

Unnamed: 0             439
City                   439
Country                439
Lat                    439
Lng                    439
Max Temp               439
Humidity               439
Cloudiness             439
Wind Speed             439
Current Description    439
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.95,light rain,3.0707,172.7902,
3,Najran,SA,88.45,light rain,17.4924,44.1277,
4,Lompoc,US,67.93,overcast clouds,34.6391,-120.4579,
9,Rikitea,PF,72.23,light rain,-23.1203,-134.9692,
11,Bambous Virieux,MU,70.21,broken clouds,-20.3428,57.7575,
12,Mahebourg,MU,71.26,broken clouds,-20.4081,57.7000,
16,Puerto Colombia,CO,87.89,overcast clouds,10.9878,-74.9547,
18,Ponta Do Sol,PT,78.12,clear sky,32.6667,-17.1000,
19,Kapaa,US,75.18,broken clouds,22.0752,-159.3190,
22,Cravo Norte,CO,85.15,broken clouds,6.3017,-70.2041,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.95,light rain,3.0707,172.7902,Isles Sunset Lodge
3,Najran,SA,88.45,light rain,17.4924,44.1277,محمد هادي ال فطيح
4,Lompoc,US,67.93,overcast clouds,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
9,Rikitea,PF,72.23,light rain,-23.1203,-134.9692,People ThankYou
11,Bambous Virieux,MU,70.21,broken clouds,-20.3428,57.7575,Casa Tia Villa
12,Mahebourg,MU,71.26,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
16,Puerto Colombia,CO,87.89,overcast clouds,10.9878,-74.9547,Hotel El Emigrante
18,Ponta Do Sol,PT,78.12,clear sky,32.6667,-17.1000,Hotel do Campo
19,Kapaa,US,75.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
22,Cravo Norte,CO,85.15,broken clouds,6.3017,-70.2041,Hotel Potosí Los Mirtos


In [10]:
# 8a. Create the output File (CSV)
output_data_file = ('WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
current_description = hotel_df["Current Description"]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))